<a href="https://colab.research.google.com/github/Besuf/Llava-colab/blob/main/Llava_Olama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# Start/restart Ollama and wait for port 11434 to be ready
!pkill -f "ollama serve" || true
%env OLLAMA_NUM_GPU=1
%env OLLAMA_KEEP_ALIVE=30m
!nohup ollama serve > /content/ollama.log 2>&1 &

import time, requests, os

BASE = "http://127.0.0.1:11434"

def wait_for_ollama(timeout=120):
    t0 = time.time()
    while time.time() - t0 < timeout:
        try:
            r = requests.get(f"{BASE}/api/version", timeout=2)
            if r.ok:
                print("Ollama up:", r.json())
                return True
        except Exception:
            pass
        time.sleep(1)
    print("Timeout waiting for Ollama. Last logs:")
    os.system("tail -n 80 /content/ollama.log")
    return False

assert wait_for_ollama(), "Ollama server did not start"

In [ ]:
!ollama pull llava:7b

In [ ]:
import requests, json

BASE = "http://127.0.0.1:11434"

# 1) Confirm server responding
ver = requests.get(f"{BASE}/api/version", timeout=5)
ver.raise_for_status()

# 2) Confirm model tag present
tags = requests.get(f"{BASE}/api/tags", timeout=10).json()
assert any(m.get("name","").startswith("llava:7b") for m in tags.get("models", [])), "llava:7b not found in tags"

# 3) Simple chat test (text only)
payload = {
    "model": "llava:7b",
    "messages": [{"role": "user", "content": "Say: ready."}],
    "stream": False
}
resp = requests.post(f"{BASE}/api/chat", json=payload, timeout=180)
resp.raise_for_status()

# Handle single JSON or accidental NDJSON
try:
    data = resp.json()
except Exception:
    lines = [l for l in resp.text.splitlines() if l.strip()]
    objs = []
    for l in lines:
        try:
            objs.append(json.loads(l))
        except:
            pass
    assert objs, "No parseable JSON in response"
    data = objs[-1]

text = data.get("message", {}).get("content", "").strip()
assert text, "Empty content from llava:7b"

print(f"llava:7b chat OK on 11434 -> {text}")

In [ ]:
# ...existing code...
import base64, requests, os
from IPython.display import Markdown, display, Image as IPyImage

BASE = "http://127.0.0.1:11434"

def b64_image(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def caption_and_tags(image_path, temperature=0.1):
    # Ask for plain text with two lines
    # Ask for plain text with two lines
    system = (
        "You are an image captioning assistant that writes short, precise, and elegant descriptions of photos.\n"
        "Your task is to describe the main subject and visual details that some value to the image.\n"
        "Do NOT mention lighting, ambiance, mood, atmosphere, or emotions unless they are essential to understanding the scene.\n"
        "Use participial phrases only when they genuinely help clarity, not to fabricate a cinematic tone.\n"
        "Caption: <one or two concise sentence as needed>\n"
    )
    user = "Describe the image and generate tags. Follow the format exactly."
    payload = {
        "model": "llava:7b",
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": user, "images": [b64_image(image_path)]},
        ],
        "stream": False,
        "options": {"temperature": temperature},
    }
    r = requests.post(f"{BASE}/api/chat", json=payload, timeout=600)
    r.raise_for_status()
    return r.json().get("message", {}).get("content", "").strip()

for img_path in ["sample-1.jpeg", "sample-2.jpeg", "sample-3.jpeg", "sample-4.jpeg", "sample-5.jpeg", "sample-6.jpeg", "sample-7.jpeg", "sample-8.jpeg"]:
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Image not found: {img_path}. CWD: {os.getcwd()}")
    display(IPyImage(filename=img_path))
    out = caption_and_tags(img_path)
    display(Markdown(out.replace("\n", "  \n")))